In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import os

In [ ]:
DATA_FOLDER = Path("../data/Inrix-Daten/date=2024-02-02")

list(DATA_FOLDER.glob("*"))

### Step 1. Undertsand the difference between reports

In [ ]:
trips_172545_path = DATA_FOLDER.joinpath("reportId=172545/v1/data/trips/trips.csv/trips.csv")

In [ ]:
df = pd.read_csv(trips_172545_path,
                 header=None,
                 escapechar="\\",  # this allows to read "bad_lines" with `{"VT":"HGV","FT":"DIESEL"}` on the last col
                 on_bad_lines="warn")

In [ ]:
df.shape

In [ ]:
df.iloc[36878:36885, :]

In [ ]:
df[df.columns[-1]].value_counts()

Given the small amount of data in this column, we will remove it.

Also, given that in `TripBulkReportTripsHeaders.csv` there are only 31 elements, I assume this is the "misplaced" column.

In [ ]:
df = df.drop(columns=[31])

headers_cols = pd.read_csv(DATA_FOLDER.joinpath("reportId=172545/v1/schema/TripBulkReportTripsHeaders.csv")).columns

df.columns = headers_cols

In [ ]:
def read_trips_csv(report_folder: Path) -> pd.DataFrame:
    
    data_file_path = report_folder.joinpath("v1/data/trips/trips.csv/trips.csv")
    headers_file_path = report_folder.joinpath("v1/schema/TripBulkReportTripsHeaders.csv")
    
    temp_df = pd.read_csv(data_file_path,
                 header=None,
                 escapechar="\\",  # this allows to read "bad_lines" with `{"VT":"HGV","FT":"DIESEL"}` on the last col
                 on_bad_lines="warn")
    
    temp_headers_cols = pd.read_csv(headers_file_path).columns

    temp_df = temp_df.drop(columns=[31])
    temp_df.columns = temp_headers_cols
    
    return temp_df


In [ ]:
df_172545 = read_trips_csv(DATA_FOLDER.joinpath("reportId=172545"))
df_172546 = read_trips_csv(DATA_FOLDER.joinpath("reportId=172546"))
df_172547 = read_trips_csv(DATA_FOLDER.joinpath("reportId=172547"))

#### Assumption 1: Start/End Date

In [ ]:
testing_column = "StartDate"

print(f"""Min and Max {testing_column} for 5: {df_172545[testing_column].min()} - {df_172545[testing_column].max()}""")
print(f"""Min and Max {testing_column} for 6: {df_172546[testing_column].min()} - {df_172546[testing_column].max()}""")
print(f"""Min and Max {testing_column} for 7: {df_172547[testing_column].min()} - {df_172547[testing_column].max()}""")

In [ ]:
testing_column = "EndDate"

print(f"""Min and Max {testing_column} for 5: {df_172545[testing_column].min()} - {df_172545[testing_column].max()}""")
print(f"""Min and Max {testing_column} for 6: {df_172546[testing_column].min()} - {df_172546[testing_column].max()}""")
print(f"""Min and Max {testing_column} for 7: {df_172547[testing_column].min()} - {df_172547[testing_column].max()}""")

It seems quite plausible that each report is generated on a per-month basis, that is, sorting by `StartDate`

### Step 2. Explore trips data

Given the size of the trip data we can merge them into one for aggregated processing and visualization.

In [ ]:
df_trips = pd.concat([df_172545, df_172546, df_172547],
                     axis=0,
                     ignore_index=True)

#  Save top parquet to later be able to load directly
if "processed" not in os.listdir("../data"):
    os.mkdir("../data/processed")
    
df_trips.to_parquet("../data/processed/all_trips.parquet")

Load saved data

In [ ]:
df_trips = pd.read_parquet("../data/processed/all_trips.parquet")

#### Look into the data

In [ ]:
df_trips["Mode"].value_counts()

GeospatialType from the documentation:

describes the trip's geospatial intersection with the requested zones
- II - Internal-to-Internal: trips that start & end within any zones;
- IE - Internal-to-External: trips that start within any zone and end outside of any zone;
- EI - External-to-Internal: trips that start outside of any zone and end within in any zone;
- EE - External-to-External: trips that start & end outside of any zones, but were selected because of an intersection with one or more zones

In [ ]:
df_trips["GeospatialType"].value_counts()

In [ ]:
df_trips["ProviderType"].value_counts()

In [ ]:
display(df_trips["OriginZoneName"].value_counts())
display(df_trips["DestinationZoneName"].value_counts())

In [ ]:
df_trips["EndpointType"].value_counts()

In [ ]:
df_trips["TripMeanSpeedKph"].plot.hist(bins=40)


In [ ]:
df_trips["TripMaxSpeedKph"].plot.hist(bins=40)

In [ ]:
df_trips["TripDistanceMeters"].plot.hist(bins=100, log=False)

In [ ]:
import plotly.express as px

df_trips_temp = df_trips[["TripMeanSpeedKph", "TripMaxSpeedKph", "TripDistanceMeters", "GeospatialType"]].copy()

df_trips_temp["Log_TripDistanceMeters"] = np.log(df_trips_temp["TripDistanceMeters"] + 10)

df_trips_temp = df_trips_temp.sample(frac=0.1, replace=False, axis=0,
                                    #  random_state=23
                                     )

px.scatter(data_frame=df_trips_temp,
           x="TripMeanSpeedKph",
           y="TripMaxSpeedKph",
           color="Log_TripDistanceMeters",
           color_continuous_scale=px.colors.diverging.balance,
           facet_col="GeospatialType",
           category_orders={"GeospatialType": ["II", "IE", "EI", "EE"]},
           marginal_x="violin",
           hover_data=["TripDistanceMeters"],
           opacity=0.2)



In [ ]:
df_trips["WaypointFreqSec"].plot.hist(bins=50, log=True)

#### Look at the geospatial data

In [ ]:
import geopandas as gpd

gdf_trips = gpd.GeoDataFrame(data=df_trips)

gdf_trips["StartLoc"] = gpd.points_from_xy(df_trips["StartLocLon"], df_trips["StartLocLat"], crs="EPSG:4326")
gdf_trips["EndLoc"] = gpd.points_from_xy(df_trips["EndLocLon"], df_trips["EndLocLat"], crs="EPSG:4326")

gdf_trips.set_geometry("StartLoc", inplace=True)

In [ ]:
gdf_trips.set_geometry("StartLoc", inplace=True)

gdf_trips.sample(frac=0.005).explore()

In [ ]:
gdf_trips.set_geometry("EndLoc", inplace=True)

gdf_trips.sample(frac=0.005).explore()

#### Use H3 indexing

In [ ]:
H3_RESOLUTION = 5

In [ ]:
from tqdm import tqdm

tqdm.pandas()

import h3.api.numpy_int as h3

gdf_trips[f"StartLoc_H3_{H3_RESOLUTION}"] = gdf_trips.progress_apply(lambda row: h3.geo_to_h3(row["StartLoc"].x, row["StartLoc"].y, H3_RESOLUTION), axis=1)
gdf_trips[f"EndLoc_H3_{H3_RESOLUTION}"] = gdf_trips.progress_apply(lambda row: h3.geo_to_h3(row["EndLoc"].x, row["EndLoc"].y, H3_RESOLUTION), axis=1)

gdf_trips.to_parquet(f"../data/processed/all_trips_h3_{H3_RESOLUTION}.parquet")

Load saved data

In [ ]:
gdf_trips = gpd.read_parquet(f"../data/processed/all_trips_h3_{H3_RESOLUTION}.parquet")

In [ ]:
gdf_trips.head(4)

#### Look at start location (by H3 cell)

In [ ]:
gdf_trips_counts = gdf_trips[f"StartLoc_H3_{H3_RESOLUTION}"].value_counts().to_frame().reset_index()

gdf_trips_counts

In [ ]:
gdf_trips_counts.dtypes

##### Using `h3_to_geo` and `ColumnLayer`

In [ ]:
def get_geo_and_count(row):
    
    lon, lat = h3.h3_to_geo(row[f"StartLoc_H3_{H3_RESOLUTION}"])
    
    # return [lat, lon, row["count"]]
    return {"lat": lat, "lon": lon} #, "count": row["count"]}

gdf_trips_counts[["lat", "lon"]] = gdf_trips_counts.progress_apply(lambda row: get_geo_and_count(row), axis=1, result_type="expand")

In [ ]:
gdf_trips_counts

In [ ]:
gdf_trips_counts.dtypes

In [ ]:
import pydeck as pdk

column_layer = pdk.Layer(
    "ColumnLayer",
    data=gdf_trips_counts,
    get_position=["lon", "lat"],
    get_elevation="count / 100",
    elevation_scale=100,
    radius=1800,
    get_fill_color=["count * 10", "count", "count * 10", 140],
    pickable=True,
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50,
                           longitude=8,
                           zoom=6.5,
                           bearing=0,
                           pitch=60)


# Render
r = pdk.Deck(layers=[column_layer],
             initial_view_state=view_state,
             tooltip={"text": "Count: {count}"})
r #.to_html("h3_centroids_ColumnLayer.html")

##### Using `h3_to_geo_boundary`, `shapely.Polygon` (for `geopandas` native) and `PolygonLayer` (for `pydeck`)

In [ ]:
import shapely

import geopandas as gpd


def get_geo_and_count(row):
    
    h3_code = np.int64(row[f"StartLoc_H3_{H3_RESOLUTION}"])  ## disapointingly necessary bc for some reason .apply casts to float and messes up the value
    
    print((row[f"StartLoc_H3_{H3_RESOLUTION}"]))
    boundary = h3.h3_to_geo_boundary(h3_code)
    polygon = shapely.Polygon(boundary)
    
    return {"boundary": boundary, "polygon": polygon}

gdf_trips_counts[["boundary", "polygon"]] = gdf_trips_counts.apply(lambda row: get_geo_and_count(row), axis=1, result_type="expand")

gdf_trips_counts = gpd.GeoDataFrame(gdf_trips_counts, geometry="polygon", crs="EPSG:4326")

In [ ]:
gdf_trips_counts

In [ ]:
import pydeck as pdk

polygon_layer = pdk.Layer(
    "PolygonLayer",
    data=gdf_trips_counts,
    get_polygon="boundary",
    get_elevation="count / 100",
    extruded=True,
    elevation_scale=100,
    wireframe=True,
    get_line_color=[0, 0, 0, 100],
    get_fill_color=["count * 10", "count", "count * 10", 140],
    pickable=True,
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50,
                           longitude=8,
                           zoom=5,
                           bearing=0,
                           pitch=60)


# Render
r = pdk.Deck(layers=[polygon_layer],
             initial_view_state=view_state,
             map_style=pdk.map_styles.CARTO_ROAD,
             tooltip={"html": """<b>Lat, Lon:</b> {lat}, {lon} <br /><b>Count:</b> {count}"""})
r #.to_html("h3_centroids_ColumnLayer.html")

##### Using `HeatmapLayer` on all data

In [ ]:
gdf_trips.head()

In [ ]:
import pydeck as pdk


COLOR_BREWER_MY_COLOR = [  ## https://colorbrewer2.org/#type=diverging&scheme=PuOr&n=10
    [127 , 59  , 8],
    [179 , 88  , 6],
    [224 , 130 , 20],
    [253 , 184 , 99],
    [254 , 224 , 182],
    [216 , 218 , 235],
    [178 , 171 , 210],
    [128 , 115 , 172],
    [84  , 39  , 136],
    [45  , 0   , 75],
]

heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_trips[["StartLocLon", "StartLocLat"]].sample(frac=0.5),
    opacity=0.7,
    intensity=12,
    get_position=["StartLocLon", "StartLocLat"],
    aggregation=pdk.types.String("SUM"),
    color_range=COLOR_BREWER_MY_COLOR,
    # threshold=1,
    # get_weight="count",
    pickable=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50,
                           longitude=8,
                           zoom=5,
                           bearing=15,
                           pitch=0)

# Render
r = pdk.Deck(layers=[heatmap_layer],
             initial_view_state=view_state,
             # map_style=pdk.map_styles.CARTO_ROAD,
             # tooltip={"html": """<b>Lat, Lon:</b> {lat}, {lon} <br /><b>Count:</b> {count}"""},
            )
r #.to_html("h3_centroids_ColumnLayer.html")